## Name: Gurunishal Saravanan
## Student ID: 801430631

# Method A

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

train_data = np.loadtxt('/content/ECG5000_TRAIN.txt')
test_data = np.loadtxt('/content/ECG5000_TEST.txt')

X_train = train_data[:, 1:]
y_train = train_data[:, 0] - 1

X_test = test_data[:, 1:]
y_test = test_data[:, 0] - 1

X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(2)
y_train = torch.tensor(y_train, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(2)
y_test = torch.tensor(y_test, dtype=torch.long)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: torch.Size([500, 140, 1])
Test shape: torch.Size([4500, 140, 1])


In [2]:
class ECGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

trn_dataset = ECGDataset(X_train, y_train)
tst_dataset = ECGDataset(X_test, y_test)

train_loader = DataLoader(trn_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(tst_dataset, batch_size=32, shuffle=False)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class ECGCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(32, 64, 5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(2)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.global_pool(x).squeeze(2)
        x = self.fc(x)
        return x

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ECGCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    print(f"Epoch {epoch+1}, Loss: {running_loss/total:.4f}, Accuracy: {correct/total:.4f}")

Epoch 1, Loss: 0.9871, Accuracy: 0.7860
Epoch 2, Loss: 0.4439, Accuracy: 0.9080
Epoch 3, Loss: 0.3510, Accuracy: 0.9160
Epoch 4, Loss: 0.3092, Accuracy: 0.9280
Epoch 5, Loss: 0.2676, Accuracy: 0.9360
Epoch 6, Loss: 0.2464, Accuracy: 0.9360
Epoch 7, Loss: 0.2236, Accuracy: 0.9380
Epoch 8, Loss: 0.2148, Accuracy: 0.9340
Epoch 9, Loss: 0.2029, Accuracy: 0.9400
Epoch 10, Loss: 0.1945, Accuracy: 0.9440
Epoch 11, Loss: 0.1715, Accuracy: 0.9500
Epoch 12, Loss: 0.1653, Accuracy: 0.9600
Epoch 13, Loss: 0.1577, Accuracy: 0.9540
Epoch 14, Loss: 0.1505, Accuracy: 0.9580
Epoch 15, Loss: 0.1456, Accuracy: 0.9580
Epoch 16, Loss: 0.1310, Accuracy: 0.9620
Epoch 17, Loss: 0.1442, Accuracy: 0.9600
Epoch 18, Loss: 0.1338, Accuracy: 0.9580
Epoch 19, Loss: 0.1278, Accuracy: 0.9620
Epoch 20, Loss: 0.1118, Accuracy: 0.9680


In [5]:
from sklearn.metrics import f1_score

model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)

        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

accuracy = correct / total
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.9316
F1 Score: 0.9263


# Method B

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from collections import Counter
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

def load_ecg5000(train_file, test_file):
    train_data = np.loadtxt(train_file)
    test_data = np.loadtxt(test_file)

    X_train = train_data[:, 1:]
    y_train = train_data[:, 0].astype(int)
    X_test = test_data[:, 1:]
    y_test = test_data[:, 0].astype(int)

    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    y_train -= 1
    y_test -= 1

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_ecg5000(
    "/content/ECG5000_TRAIN.txt",
    "/content/ECG5000_TEST.txt"
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (500, 140, 1), Test shape: (4500, 140, 1)


In [18]:
def build_fcn(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(128, 8, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(256, 5, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def residual_block(x, filters, kernel_size=8, stride=1):
    shortcut = x
    x = layers.Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, 1, padding='same')(shortcut)
    x = layers.add([shortcut, x])
    x = layers.Activation('relu')(x)
    return x

def build_resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = inputs
    for filters in [64, 128, 128]:
        x = residual_block(x, filters)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_encoder(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(128, 5, padding='same', activation='relu')(inputs)
    x = layers.LayerNormalization()(x)
    x = layers.Conv1D(256, 11, padding='same', activation='relu')(x)
    x = layers.LayerNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_mlp(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Flatten()(inputs)
    x = layers.Dense(500, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(500, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [19]:
def ensemble_predict(models, X):
    preds = [np.argmax(m.predict(X), axis=1) for m in models]
    preds = np.stack(preds, axis=1)
    final_pred = np.array([Counter(row).most_common(1)[0][0] for row in preds])
    return final_pred

In [21]:
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

models_list = [
    build_fcn(input_shape, num_classes),
    build_resnet(input_shape, num_classes),
    build_encoder(input_shape, num_classes),
    build_mlp(input_shape, num_classes)
]

for model in models_list:
    print(f"Training {model.name}...")
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=2)

Training functional_10...
Epoch 1/10
8/8 - 7s - 872ms/step - accuracy: 0.8340 - loss: 0.8310
Epoch 2/10
8/8 - 5s - 581ms/step - accuracy: 0.9040 - loss: 0.4721
Epoch 3/10
8/8 - 3s - 426ms/step - accuracy: 0.9220 - loss: 0.3734
Epoch 4/10
8/8 - 5s - 631ms/step - accuracy: 0.9320 - loss: 0.3035
Epoch 5/10
8/8 - 4s - 470ms/step - accuracy: 0.9340 - loss: 0.2611
Epoch 6/10
8/8 - 4s - 508ms/step - accuracy: 0.9440 - loss: 0.2207
Epoch 7/10
8/8 - 7s - 850ms/step - accuracy: 0.9420 - loss: 0.2137
Epoch 8/10
8/8 - 4s - 558ms/step - accuracy: 0.9500 - loss: 0.2008
Epoch 9/10
8/8 - 3s - 420ms/step - accuracy: 0.9460 - loss: 0.1935
Epoch 10/10
8/8 - 3s - 335ms/step - accuracy: 0.9540 - loss: 0.1781
Training functional_11...
Epoch 1/10
8/8 - 13s - 2s/step - accuracy: 0.8320 - loss: 0.6958
Epoch 2/10
8/8 - 11s - 1s/step - accuracy: 0.9280 - loss: 0.2562
Epoch 3/10
8/8 - 10s - 1s/step - accuracy: 0.9380 - loss: 0.2180
Epoch 4/10
8/8 - 13s - 2s/step - accuracy: 0.9440 - loss: 0.1980
Epoch 5/10
8/8 - 

In [23]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = ensemble_predict(models_list, X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Ensemble accuracy: {acc:.4f}")
print(f"Ensemble F1-score (macro): {f1:.4f}")

141/141 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step
141/141 ━━━━━━━━━━━━━━━━━━━━ 13s 94ms/step
141/141 ━━━━━━━━━━━━━━━━━━━━ 11s 74ms/step
141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Ensemble accuracy: 0.9009
Ensemble F1-score (macro): 0.8780


# Method C

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

train_data = np.loadtxt('/content/ECG5000_TRAIN.txt')
test_data = np.loadtxt('/content/ECG5000_TEST.txt')

X_train = train_data[:, 1:]
y_train = train_data[:, 0] - 1

X_test = test_data[:, 1:]
y_test = test_data[:, 0] - 1

# Initialize KNN
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')

# Train
knn.fit(X_train, y_train)

# Predict
y_pred = knn.predict(X_test)


from sklearn.metrics import accuracy_score, f1_score

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)

# Compute F1-score (macro average)
f1 = f1_score(y_test, y_pred, average='weighted')

# Print both
print(f"KNN Accuracy: {accuracy*100:.2f}%")
print(f"KNN F1-score (macro): {f1:.4f}")

KNN Accuracy: 93.49%
KNN F1-score (macro): 0.9264
